# Como um Agent é construído

A ideia central dos agentes é usar um modelo de linguagem para escolher uma sequência de ações a serem executadas. Em cadeias, uma sequência de ações é codificada diretamente (no código). Em agentes, um modelo de linguagem é usado como um motor de raciocínio para determinar quais ações devem ser tomadas e em qual ordem.

## Criando as tools que usaremos

In [1]:
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())

In [15]:
import requests
import datetime
import wikipedia
from langchain.agents import tool
from pydantic import BaseModel, Field

class ArgsTemp(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool
def busca_temperatura_atual(latitude: float, longitude: float):
    """Retorna a temperatura atual de uma determinada localidade"""
    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x]-hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f"{temp_atual}°C"
    else:
        raise Exception(f'Request para a API {URL} falhou: {resposta.status_code}')
    
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz uma busca na Wikipédia de uma query e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo} \nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return "Busca não teve retorno"
    else:
        return '\n\n'.join(resumos)

## Revisando a utilização das tools

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável e gentil chamado Mimir'),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')
tools = [busca_temperatura_atual, busca_wikipedia]
tools_json = [convert_to_openai_function(tool)for tool in tools]
tools_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions = tools_json)

In [17]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions = tools_json) | OpenAIFunctionsAgentOutputParser()

In [18]:
resposta = chain.invoke({'input': 'Qual a temperatura em Catalão Goiás?'})

In [19]:
resposta.tool

'busca_temperatura_atual'

In [20]:
resposta.tool_input

{'latitude': -18.1698, 'longitude': -47.9382}

In [21]:
resposta.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-18.1698,"longitude":-47.9382}', 'name': 'busca_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 110, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-c44c190a-b08b-4164-a9a6-07617cf46633-0', usage_metadata={'input_tokens': 110, 'output_tokens': 31, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

## Adicionando o racicínio do agent as mensagens (agent_scretchpad)

Temos que adicionar junto as nossas mensagens um campo que armazenará o raciocínio atual do modelo chamado agent_scratchpad. Para isso, utilizamos um MessagesPlaceholder ao nosso prompt. Ele guardará espaço para o raciocínio e, caso o modelo não esteja gerando um raciocínio no momento, o MessagesPlaceholder não é utilizado.

In [22]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável e gentil chamado Mimir'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

chain = prompt | chat.bind(functions = tools_json) | OpenAIFunctionsAgentOutputParser()

In [23]:
resposta_inicial = chain.invoke({'input': 'Qual a temperatura para Catalão Goiás?',
                                 'agent_scratchpad': []})
resposta_inicial

AgentActionMessageLog(tool='busca_temperatura_atual', tool_input={'latitude': -18.1738, 'longitude': -47.9388}, log="\nInvoking: `busca_temperatura_atual` with `{'latitude': -18.1738, 'longitude': -47.9388}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-18.1738,"longitude":-47.9388}', 'name': 'busca_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 110, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-6fbff8e9-f5f4-4afd-a3ab-c1da368cb57c-0', usage_metadata={'input_tokens': 110, 'output_tokens': 31, 'total_tokens': 141, 'input_token_details': {'audio

In [24]:
observacao = tools_run[resposta_inicial.tool].run(resposta_inicial.tool_input)
observacao

'26.0°C'

Podemos utilizar a função format_to_openai_function_messages para modificar o formato da resposta de forma que ela possa ser enviada, junto da observação, de volta ao modelo. No caso o que está ocorrendo é o modelo está pedindo que uma tool seja rodada, estamos rodando a tool e gerando uma observação, e a gora enviamos novamente para o modelo a pergunta original, a mensagem do próprio modelo dizendo que precisava que a tool fosse rodada e a observação gerada pela ferramenta.

In [25]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages

format_to_openai_function_messages([(resposta_inicial, observacao)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-18.1738,"longitude":-47.9388}', 'name': 'busca_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 110, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-6fbff8e9-f5f4-4afd-a3ab-c1da368cb57c-0', usage_metadata={'input_tokens': 110, 'output_tokens': 31, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='26.0°C', additional_kwargs={}, response_metadata={}, name='busca_temperatura_atual')]

In [26]:
resposta_final = chain.invoke({'input': 'Qual a temperatura para Catalão Goiás?',
                                 'agent_scratchpad': format_to_openai_function_messages([(resposta_inicial, observacao)])})

resposta_final

AgentFinish(return_values={'output': 'A temperatura atual em Catalão, Goiás, é de 26°C.'}, log='A temperatura atual em Catalão, Goiás, é de 26°C.')

### Criando um loop de racicínio

Por fim, podemos criar um loop que adiciona a automaticamente as chamadas de função e observações e fica chamando o modelo novamente até que a mensagem de AgentFinish seja recebida.

In [30]:
from langchain.schema.agent import AgentFinish

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'agent_scratchpad': format_to_openai_function_messages(passos_intermediarios)
            })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tools_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

In [31]:
run_agent('Qual é a temperatura para Catalão Goiás?')

AgentFinish(return_values={'output': 'A temperatura atual em Catalão, Goiás, é de 26,0 °C.'}, log='A temperatura atual em Catalão, Goiás, é de 26,0 °C.')

Modificamos um pouco o formato para padronizar ao funcionamento do LangChain para agents

In [32]:
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough

pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)

chain = pass_through | prompt | chat.bind(functions=tools_json) |  OpenAIFunctionsAgentOutputParser()

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
            })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tools_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

In [34]:
pass_through.invoke({'input': 'Qual a temperatura de Floripa?', 'intermediate_steps': []})

{'input': 'Qual a temperatura de Floripa?',
 'intermediate_steps': [],
 'agent_scratchpad': []}

In [33]:
run_agent('Qual a temperatura de Floripa?')

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 18,6°C. Se precisar de mais informações, é só avisar!'}, log='A temperatura atual em Florianópolis é de 18,6°C. Se precisar de mais informações, é só avisar!')

## O que temos no final?

### Um Agent

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável e gentil chamado Mimir'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)

chain = pass_through | prompt | chat.bind(functions=tools_json) |  OpenAIFunctionsAgentOutputParser()

### Um AgentExecutor

In [ ]:
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
            })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tools_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))